In [3]:
# Use Braket SDK Cost Tracking to estimate the cost to run this example
from braket.tracking import Tracker
t = Tracker().start()


# general imports
import matplotlib.pyplot as plt
%matplotlib inline

# AWS imports: Import Braket SDK modules
from braket.circuits import Circuit, Gate, Instruction, circuit, Observable
from braket.devices import LocalSimulator
from braket.aws import AwsDevice ,AwsQuantumTask
import numpy as np

import json     

In [30]:

#basic non computational datas about the task
def list_task_metadata( qtask):
    meta = qtask.metadata()

    params = meta["deviceParameters"]

    print("############################")
    print("Quantum Task Metadata")
    print("Task arn:", meta["quantumTaskArn"])
    print("Task shots#:", meta["shots"])
    print("Task status:", meta["status"])
    print("Task tags:", meta["tags"])
    print("Task on device:", meta["deviceArn"])
    print("Creation date:", meta["createdAt"])
    print("End date:", meta["endedAt"])
    print("Output buket data:", meta["outputS3Directory"], " / ", meta["outputS3Bucket"])
    
    # for key in meta.keys() :
    #     print( key)

# HTTP related data for the exchange protocol(?)
def list_task_response_metadata(qtask):
    meta = qtask.metadata()

    response = meta["ResponseMetadata"]

    for key  in response.keys() :
        print( key)

def list_task_device_metadata(qtask):
    meta = qtask.metadata()

    dev = meta["deviceParameters"]
    devDict = json.loads(dev)

    schema = devDict["braketSchemaHeader"]
    parag = devDict["paradigmParameters"]

    print(schema["version"])
    print(schema["name"])

    for key, values in schema :
        print( key, values)

In [29]:
#retrieve example Simulator Task

sim_task_id = "arn:aws:braket:eu-west-2:058017682654:quantum-task/3a9794d5-58b7-48f2-93da-b3228c89513d" 
task = AwsQuantumTask(arn = sim_task_id)

list_task_device_metadata(task)

1
braket.device_schema.simulators.gate_model_simulator_device_parameters
braket.device_schema.simulators.gate_model_simulator_device_parameters
1
